# Solving a mixed Neumann-Dirichlet Problem

### Background

With BEM++ it is possible to define operators only on segments of a given domain. This makes it possible to solve mixed Neumann-Dirichlet problems. In the following we solve the Laplace equation inside the unit cube with unit Dirichlet boundary conditions on two sides and zero Neumann boundary conditions on the other four sides.

Denote by $\Gamma_D$ the part of the boundary that holds the Dirichlet boundary conditions and by $\Gamma_N$ the boundary part that holds the Neumann boundary conditions. We denote by $t\in\Gamma_D$ the unkown Neumann data and by $u\in\Gamma_N$ the unknown Dirichlet data. The given Dirichlet data on $\Gamma_D$ is denoted by $g_D$ and the given Neumann data on $\Gamma_N$ is denoted by $g_N$.

From Green's representation theorem it follows that
$$
\begin{align}
\left[Vt\right](x) - \left[Ku\right](x) &= \left[\frac{1}{2}I + K\right]g_D(x) - Vg_N(x),\quad x\in\Gamma_D\\
\left[Du\right](x) + \left[K't\right](x) &=\left[\frac{1}{2}I - K'\right]g_N(x) - Dg_D(x),\quad x\in\Gamma_N
\end{align}
$$
Here, as usual $V$ is the single layer, $K$ the double layer, $K'$ the adjoint double layer and $D$ the hypersingular boundary operator.

The difficulty in the implementation is the definition of the discrete function spaces and the treatment of dofs that lie on the interface between $\Gamma_N$ and $\Gamma_D$. In the following we will go through the implementation and point out how to correctly define all spaces involved.

### Implementation

We start with the usual imports. In addition we increase the integration order as in this example we will be working with spaces of quadratic functions.

In [16]:
import bempp.api
import numpy as np

bempp.api.global_parameters.quadrature.medium.double_order = 4
bempp.api.global_parameters.quadrature.far.double_order = 4

We now define the domain. We use a standard unit cube. In the corresponding function all sides of the cube are already associated with different domain indices. We associate the indices 1 and 3 with the Dirichlet boundary and the other indices with the neumann boundary.

In [17]:
grid = bempp.api.shapes.cube()
dirichlet_segments = [1, 3]
neumann_segments = [2, 4, 5, 6]

We can now define the spaces. For the Neumann data we use discontinuous basis functions order 1. For the Dirichlet data we use continuous basis functions of local polynomial order 2.

We need global spaces for the Dirichlet and Neumann data and suitable spaces on the segments. In the following we list the space definitions.

* The ``neumann_space_dirichlet_segment`` space holds the unknown Neumann data $t$ on $\Gamma_D$. For $\Gamma_D$ we use the parameter ``closed=True``, meaning that all boundary edges and the associated dofs on the boundary edges are part of the space. The parameter ``element_on_segment=True`` implies that we restrict functions to elements that lie on elements associated with $\Gamma_D$. This is important for dofs on boundary edges and excludes associated functions that lie just outside $\Gamma_D$ on the other side of the boundary edge.

* The ``neumann_space_neumann_segment`` space is defined on $\Gamma_N$. $\Gamma_N$ is open, that is boundary edges are not part of the space. We again restrict basis functions to $\Gamma_N$ by the parameter ``element_on_segment=True``. However, we also include all functions which are defined on elements of the space but whose reference points (i.e. the dof positions) are on the excluded boundary. This is achieved by the parameter ``reference_point_on_segment=False``. If it were set to true (default) it would only include dofs whose reference points lie in the segment and not on the excluded boundary.

* The ``dirichlet_space_dirichlet_segment`` space is a space of continuous basis functions that holds the Dirichlet data on $\Gamma_D$. The space is closed and by default basis functions are allowed to extend into the elements adjacent to $\Gamma_D$. This extension is necessary because of the definition of the underlying Sobolev space on the segment. To control this behavior for continuous spaces the option ``strictly_on_segment`` exists, which is by default set to false.

* The ``dirichlet_space_neumann_segment`` is defined similar to the ``dirichlet_space_dirichlet_segment`` but on the open segment $\Gamma_N$.

* For the discretization of the Dirichlet data we also need the space ``dual_dirichlet_space``. This is the correct dual space for projecting functions into the space of Dirichlet data.

In [18]:

order_neumann = 1
order_dirichlet = 2


global_neumann_space = bempp.api.function_space(grid, "DP", order_neumann)
global_dirichlet_space = bempp.api.function_space(grid, "P", order_dirichlet)

neumann_space_dirichlet_segment = bempp.api.function_space(grid, "DP", order_neumann, 
                                                           domains=dirichlet_segments, closed=True,
                                                          element_on_segment=True)
                                    

neumann_space_neumann_segment = bempp.api.function_space(grid, "DP", order_neumann,
                                                         domains=neumann_segments, closed=False,
                                                        element_on_segment=True,
                                                        reference_point_on_segment=False)

dirichlet_space_dirichlet_segment = bempp.api.function_space(grid, "P", order_dirichlet,
                                                            domains=dirichlet_segments, closed=True)

dirichlet_space_neumann_segment = bempp.api.function_space(grid, "P", order_dirichlet,
                                                            domains=neumann_segments, closed=False)

dual_dirichlet_space = bempp.api.function_space(grid, "P", order_dirichlet, domains=dirichlet_segments,
                                               closed=True, strictly_on_segment=True)


In the following we define all operators on the corresponding spaces and the overall block operator.

In [19]:
slp_DD = bempp.api.operators.boundary.laplace.single_layer(neumann_space_dirichlet_segment,
                                                          dirichlet_space_dirichlet_segment,
                                                          neumann_space_dirichlet_segment)

dlp_DN = bempp.api.operators.boundary.laplace.double_layer(dirichlet_space_neumann_segment,
                                                          dirichlet_space_dirichlet_segment,
                                                          neumann_space_dirichlet_segment)

adlp_ND = bempp.api.operators.boundary.laplace.adjoint_double_layer(neumann_space_dirichlet_segment,
                                                                   neumann_space_neumann_segment,
                                                                   dirichlet_space_neumann_segment)

hyp_NN = bempp.api.operators.boundary.laplace.hypersingular(dirichlet_space_neumann_segment,
                                                   neumann_space_neumann_segment,
                                                   dirichlet_space_neumann_segment)

slp_DN = bempp.api.operators.boundary.laplace.single_layer(neumann_space_neumann_segment,
                                                  dirichlet_space_dirichlet_segment,
                                                  neumann_space_dirichlet_segment)

dlp_DD = bempp.api.operators.boundary.laplace.double_layer(dirichlet_space_dirichlet_segment,
                                                  dirichlet_space_dirichlet_segment,
                                                  neumann_space_dirichlet_segment)

id_DD = bempp.api.operators.boundary.sparse.identity(dirichlet_space_dirichlet_segment,
                                            dirichlet_space_dirichlet_segment,
                                            neumann_space_dirichlet_segment)

adlp_NN = bempp.api.operators.boundary.laplace.adjoint_double_layer(neumann_space_neumann_segment,
                                                           neumann_space_neumann_segment,
                                                           dirichlet_space_neumann_segment)

id_NN = bempp.api.operators.boundary.sparse.identity(neumann_space_neumann_segment,
                                                    neumann_space_neumann_segment,
                                                    dirichlet_space_neumann_segment)

hyp_ND = bempp.api.operators.boundary.laplace.hypersingular(dirichlet_space_dirichlet_segment,
                                                   neumann_space_neumann_segment,
                                                   dirichlet_space_neumann_segment)

blocked = bempp.api.BlockedOperator(2, 2)

blocked[0, 0] = slp_DD
blocked[0, 1] = -dlp_DN
blocked[1, 0] = adlp_ND
blocked[1, 1] = hyp_NN

In the following we define the functions of the Dirichlet and Neumann data and their discretizations on the corresponding segments.

In [20]:
def dirichlet_data_fun(x):
    return 1
    
def dirichlet_data(x, n, domain_index, res):
    res[0] = dirichlet_data_fun(x)
    
def neumann_data_fun(x):
    return 0
 
def neumann_data(x, n, domain_index, res):
    res[0] = neumann_data_fun(x)

dirichlet_grid_fun = bempp.api.GridFunction(dirichlet_space_dirichlet_segment,
                                            fun=dirichlet_data,
                                            dual_space=dual_dirichlet_space)

neumann_grid_fun = bempp.api.GridFunction(neumann_space_neumann_segment,
                                         fun=neumann_data,
                                         dual_space=dirichlet_space_neumann_segment)

rhs_fun1 = (-.5 * id_DD + dlp_DD) * dirichlet_grid_fun - slp_DN * neumann_grid_fun
rhs_fun2 = -hyp_ND * dirichlet_grid_fun + (-.5 * id_NN - adlp_NN) * neumann_grid_fun

INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,1037). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 7.73E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2934,1897). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 7.78E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,1464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 8.29E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,1464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 7.03E-03 sec.
INFO:BEMPP:DLP. START ASSEMBLY. Dim: (1037,1464). Assembly Type: hmat
INFO:BEMPP:DLP. FINISHED ASSEMBLY. Time: 6.68E-01 sec. Mem Size (Mb): 1.52E+00. Compression: 1.31E-01
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,1464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 6.79E-03 sec.
INFO:BEMPP:SLP. START ASSEMBLY. Dim: (2934,1464). Assembly Type: hmat
INFO:BEMPP:SLP. FINISHED ASSEMBLY. Time: 4.68E-01 sec. Mem Size (Mb): 1.9

We can now discretize and solve the block operator system. We solve without preconditioner. This causes problems if we further increase the degree of the basis functions.

In [21]:
# Discretize and solve the system

lhs = blocked.weak_form()
rhs = np.hstack([rhs_fun1.projections(neumann_space_dirichlet_segment), 
                 rhs_fun2.projections(dirichlet_space_neumann_segment)])

from scipy.sparse.linalg import gmres
x, info = gmres(lhs, rhs)

INFO:BEMPP:SLP. START ASSEMBLY. Dim: (1464,1464). Assembly Type: hmat
INFO:BEMPP:SLP. FINISHED ASSEMBLY. Time: 4.73E-01 sec. Mem Size (Mb): 2.61E+00. Compression: 1.60E-01
INFO:BEMPP:DLP. START ASSEMBLY. Dim: (1897,1464). Assembly Type: hmat
INFO:BEMPP:DLP. FINISHED ASSEMBLY. Time: 7.38E-01 sec. Mem Size (Mb): 2.16E+00. Compression: 1.02E-01
INFO:BEMPP:ADJ_DLP. START ASSEMBLY. Dim: (1464,1897). Assembly Type: hmat
INFO:BEMPP:ADJ_DLP. FINISHED ASSEMBLY. Time: 7.71E-01 sec. Mem Size (Mb): 2.16E+00. Compression: 1.02E-01
INFO:BEMPP:HYP. START ASSEMBLY. Dim: (1897,1897). Assembly Type: hmat
INFO:BEMPP:HYP. FINISHED ASSEMBLY. Time: 5.07E+00 sec. Mem Size (Mb): 5.33E+00. Compression: 1.94E-01
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,1464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.44E-02 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2934,1897). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.24E-02 sec.


In the following we split up the solution vector and define the grid functions associated with the computed Neumann and Dirichlet data.

In [22]:
nx0 = neumann_space_dirichlet_segment.global_dof_count
neumann_solution = bempp.api.GridFunction(neumann_space_dirichlet_segment, coefficients=x[:nx0])
dirichlet_solution = bempp.api.GridFunction(dirichlet_space_neumann_segment, coefficients=x[nx0:])

We want to recombine the computed Dirichlet and Neumann data with the corresponding known data in order to get Dirichlet and Neumann grid functions defined on the whole grid. To achieve this we define identity operators from $\Gamma_N$ and $\Gamma_D$ into the global Dirichlet and Neumann spaces.

In [23]:
neumann_imbedding_dirichlet_segment = bempp.api.operators.boundary.sparse.identity(neumann_space_dirichlet_segment,
                                                            global_neumann_space,
                                                            global_neumann_space)

neumann_imbedding_neumann_segment = bempp.api.operators.boundary.sparse.identity(neumann_space_neumann_segment,
                                                                            global_neumann_space,
                                                                            global_neumann_space)

dirichlet_imbedding_dirichlet_segment = bempp.api.operators.boundary.sparse.identity(dirichlet_space_dirichlet_segment,
                                                                                global_dirichlet_space,
                                                                                global_dirichlet_space)

dirichlet_imbedding_neumann_segment = bempp.api.operators.boundary.sparse.identity(dirichlet_space_neumann_segment,
                                                                              global_dirichlet_space,
                                                                              global_dirichlet_space)

dirichlet = (dirichlet_imbedding_dirichlet_segment * dirichlet_grid_fun +
             dirichlet_imbedding_neumann_segment * dirichlet_solution)

neumann = (neumann_imbedding_neumann_segment * neumann_grid_fun +
           neumann_imbedding_dirichlet_segment * neumann_solution)


INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2934,2934). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 9.85E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1037,2934). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 8.92E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2934,2934). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 9.01E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (1897,2934). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.01E-02 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (4398,4398). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 7.48E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2934,4398). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 8.48E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (4398,4398). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 6.45E-03 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim:

We can plot the solution using the command ``dirichlet.plot()``. The solution looks as follows. <img src="cube_mixed_solution.png">